# Exploratory and Statistical Data Analysis for Iced company

## 1. Data Preparation

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../databases/games.csv')
data.columns = data.columns.str.lower()
#print(data.dtypes)

## Convertimos tipos de datos que mejor describan la columna
data['year_of_release'] = data['year_of_release'].astype('Int64', errors='ignore') # Conversión a int
data['user_score'] = pd.to_numeric(data['user_score'], errors='coerce') # Convertimos a float

print(data.info())
#display(data)
print('Duplicados: ',data.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  Int64  
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       7590 non-null   float64
 10  rating           9949 non-null   object 
dtypes: Int64(1), float64(6), object(4)
memory usage: 1.4+ MB
None
Duplicados:  0


In [5]:
## ======================= GESTIÓN DE VALORES AUSENTES(NaN) ===============================
# --- Columna rating:
#print(data['rating'].unique()) # Vemos que tiene un valor NAN correspondiente a los juegos sin clasificación. Lo sustituimos por unrated
data['rating'].fillna('unrated', inplace=True)

# --- Columna critic_score y user_score
# Los valores ausentes se conservarán, para que la fila no pierda el tipo de dato numérico float

# --- Columnas x_sales:
# Para las columnas de ventas, los nan se reeplazaran por 0, pues son valores que no se capturaron en la base de datos, pero que de no eliminarlos, podemos crear NaN al momento de obtener las ventas totales, de modo que convirtiendolas a cero, las operaciones se pueden hacer, sin crear valores nulos ni afectar a los valores que sí se capturaron.
data['other_sales'].fillna(0, inplace=True)
data['jp_sales'].fillna(0, inplace=True)
data['eu_sales'].fillna(0, inplace=True)
data['na_sales'].fillna(0, inplace=True)

# --- Columnas name, year_of_release y genre: 
#display(data[data['name'].isna()==True])
# Los valores ausentes de estas columnas provienen de videojuegos no identificados en la base de datos, quizás por la antiguedad de los mismos. De modo que sustituiremos estos valores por la abrevietura TBD: To Be Determined.
data['name'].fillna('TBD',inplace=True)
data['genre'].fillna('TBD',inplace=True)
# En el caso de year_of_release, esta columna perderá el tipo entero y será str
#data[data['year_of_release'].isna()==True]['year_of_release']
data['year_of_release'] = data['year_of_release'].astype('str', errors='ignore')
data['year_of_release'].replace('<NA>', 'TBD', inplace=True)

# --- Visualizamos la versión del DF corregido
#display(data)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16715 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16715 non-null  object 
 3   genre            16715 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       7590 non-null   float64
 10  rating           16715 non-null  object 
 11  total_sales      16715 non-null  float64
dtypes: float64(7), object(5)
memory usage: 1.5+ MB
None


In [4]:
# Cálculo de las ventas totales:
data['total_sales'] = data['na_sales'] + data['eu_sales'] + data['jp_sales'] + data['other_sales']
display(data.head())

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E,82.54
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,unrated,40.24
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E,35.52
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E,32.77
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,unrated,31.38


## 2. Data Analysis
#### How many games were released in different years?